# Simple AI Coding Agent Using The REST API (And Some Python)

## Python Setup

Imports some modules for HTTPS requests and handling JSON

In [ ]:
import json
import os

import requests

Set the URL, Headers and API Key

In [ ]:
URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent" 

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

headers = {
    "x-goog-api-key": GEMINI_API_KEY,
    "Content-Type": "application/json",
}

## Some Utility Functions

In [ ]:
def print_answer(resp, debug=False):
    res = json.loads(resp.text)
    if debug:
        print(res)
        print()
    role = res['candidates'][0]['content']['role']
    answer = res['candidates'][0]['content']['parts'][0]['text']
    print(f"{role}: {answer}")

In [ ]:
def make_request(payload):
    response = requests.post(URL, headers=headers, data=json.dumps(payload))
    if response.status_code == 200:
        print_answer(response)
    else:
        print(f"Errror, status code: {response.status_code} details: {response.text}")

def make_request_with_tools(payload, debug=False):
    response = requests.post(URL, headers=headers, data=json.dumps(payload))
    if response.status_code == 200:
        res = json.loads(response.text)
        if debug:
            print(res)
            print()
            parts = res['candidates'][0]['content']['parts'][0]
            role = res['candidates'][0]['content']['role']
            if 'functionCall' in parts:
                print(f"Tool call: {parts['functionCall']}")
            # answer = ['text']
            # print(f"{role}: {answer}")
    else:
        print(f"Errror, status code: {response.status_code} details: {response.text}")

## A Simple Chat Request

A simple call to an LLM for a response involves sending a JSON payload in a HTTPS POST request. 

The content of the "chat" is in the 'text' section of the 'parts'. The 'contents' array is an array of objects that make up the contents of the call to the LLM.

In [ ]:
payload = {
    "contents": [
      {
        "role": "user",
        "parts": [
          {
            "text": "Explain how AI works in a few words"
          }
        ]
      }
    ]
}

make_request(payload)

## Defining Model Behaviour With A System Instruction

In [ ]:
system_instructions = {
    "system_instruction": {
      "parts": [
        {
          "text": "You are a coding agent, act like a software engineer. Your name is John's AI Coding Bot."
        }
      ]
    }
}

In [ ]:
payload = system_instructions | {
    "contents": [
      {
        "role": "user",
        "parts": [
          {
            "text": "Who are you? What do you do?"
          }
        ]
      }
    ]
}

make_request(payload)

## Add Some Tools

In [ ]:
tools = {
"tools": [
        {
            "function_declarations": [
                {
                    "name": "write_code",
                    "description": "writes some code to a file",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "filename": {
                                "type": "string",
                                "description": "The filename to write the code to"
                            },
                            "code": {
                                "type": "string",
                                "description": "The code to write to the file"
                            }
                        },
                        "required": [
                            "filename",
                            "code"
                        ]
                    }
                }
            ]
        }
    ]
}

In [ ]:
payload = system_instructions | tools | {    
    "contents": [
        {
            "role": "user",
            "parts": [
                {
                    "text": "Who are you? What do you do? What tools can you use?"
                }
            ]
        }
    ]
}

make_request(payload)

## Multiturn Work

Let's provide the write_code tool we told the LLM we have.

In [ ]:
def write_code(filename, code):
    try:
        path = os.path.join("./", filename)
        with open(path, "w") as f:
            f.write(code)
    except Exception as e:
        return False

    return True

## Ask The Agent To Write Code

In [ ]:
payload = system_instructions | tools | {    
    "contents": []
}

first_user_request = {
    "role": "user",
    "parts": [
        {
            "text": "Write a python script to output the first 10 numbers of the Fibonacci sequence."
        }
    ]
}

payload["contents"].append(first_user_request)

response = requests.post(URL, headers=headers, data=json.dumps(payload))
if response.status_code == 200:
    res = json.loads(response.text)
    print(response.text)
else:
    print(f"Errror, status code: {response.status_code} details: {response.text}")

# make_request_with_tools(payload, debug=True)

In [ ]:
tool_call = {}
parts = res['candidates'][0]['content']['parts'][0]
role = res['candidates'][0]['content']['role']
if 'functionCall' in parts:
    tool_call = res['candidates'][0]['content']
    function = parts['functionCall']['name']
    filename = parts['functionCall']['args']['filename']
    code = parts['functionCall']['args']['code']
    print(f"Tool call requested\nFunction: `{function}` with filename: `{filename}`")
    print("Provided code:")
    print(code)
if 'text' in parts:
    print(f"{role}: {parts['text']}")

## Making The Tool Call:

In [ ]:
if function == 'write_code':
    tool_call_result = write_code(filename, code)
    print(f"Writing code returned: {tool_call_result}")

In [ ]:
!cat $filename

## Creating A Response

Having called that function with the provide arguments, we then send the model a tool response based on the function output. That response would look like the following:

In [ ]:
tool_response = {
    "role": "user",
    "parts": [{
        "functionResponse": {
            "name": "write_code",
            "response": {
                "success": tool_call_result,
                "output": "File written successfully"
            }
        }
    }]
}

In [ ]:
# Next we call the model, recording it's tool call
payload["contents"].append(tool_call)

# And the tool response as new messages in the contents list
payload["contents"].append(tool_response)

# Let's see the payload
print(json.dumps(payload, indent=4))


In [ ]:
make_request(payload)

# Docs

In [ ]:
print("foo")
